## Edgar Moises Hernandez Gonzalez

In [3]:
import time

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score, confusion_matrix

In [4]:
df_train = pd.read_csv("train_set.csv")
df_train

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,trip_id,duration,start_time,end_time,start_lat,start_lon,end_lat,end_lon,bike_id,plan_duration,trip_route_category,passholder_type,start_station,end_station
0,101750280,35,2018-08-07 11:20:00,2018-08-07 11:55:00,33.748920,-118.275192,33.748920,-118.275192,06530,1.0,Round Trip,Walk-up,4127,4127
1,46560345,32,9/17/2017 17:51,9/17/2017 18:23,34.035679,-118.270813,34.047749,-118.243172,6683,0.0,One Way,Walk-up,3057,3062
2,120016336,6,2019-04-22 09:22:00,2019-04-22 09:28:00,34.046070,-118.233093,34.047749,-118.243172,06710,30.0,One Way,Monthly Pass,3022,3062
3,129547190,138,9/22/2019 11:27,9/22/2019 13:45,34.062580,-118.290092,34.059689,-118.294662,17068,1.0,One Way,One Day Pass,4304,4311
4,136619463,14,1/31/2020 17:11,1/31/2020 17:25,34.026291,-118.277687,34.021660,-118.278687,18841,30.0,One Way,Monthly Pass,4266,4443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699995,83666862,17,2018-04-25 18:32:00,2018-04-25 18:49:00,34.049889,-118.255882,34.074829,-118.258728,06505,30.0,One Way,Monthly Pass,3032,4220
699996,132547303,9,2019-11-12 16:49:10,2019-11-12 16:58:51,34.039188,-118.232529,34.056610,-118.237213,18348,30.0,One Way,Monthly Pass,3036,3014
699997,67332628,6,2018-01-13 15:49:00,2018-01-13 15:55:00,34.049889,-118.255882,34.050480,-118.254593,05718,365.0,One Way,Flex Pass,3032,3007
699998,114846232,3,2019-02-04 14:16:00,2019-02-04 14:19:00,34.045181,-118.250237,34.045540,-118.256668,12195,1.0,One Way,Walk-up,3067,3006


In [5]:
df_train = df_train[[
    "duration", "trip_route_category", "start_station", "end_station", "passholder_type"
]]
df_train

,duration,trip_route_category,start_station,end_station,passholder_type
0,35,Round Trip,4127,4127,Walk-up
1,32,One Way,3057,3062,Walk-up
2,6,One Way,3022,3062,Monthly Pass
3,138,One Way,4304,4311,One Day Pass
4,14,One Way,4266,4443,Monthly Pass
...,...,...,...,...,...
699995,17,One Way,3032,4220,Monthly Pass
699996,9,One Way,3036,3014,Monthly Pass
699997,6,One Way,3032,3007,Flex Pass
699998,3,One Way,3067,3006,Walk-up


In [6]:
# eliminar NaN por filas
df_train = df_train.dropna()

In [7]:
# index de 0 a n-1
df_train = df_train.reset_index(drop=True)

In [8]:
ohe = OneHotEncoder(sparse=False)
transformed = ohe.fit_transform(df_train[["trip_route_category"]])
ohe.categories_

[array(['One Way', 'Round Trip'], dtype=object)]

In [9]:
df_train["one_way"] = transformed[:,0]
df_train["round_trip"] = transformed[:,1]
df_train = df_train.drop(["trip_route_category"], axis=1)

In [10]:
df_train["one_way"] = df_train["one_way"].apply(lambda x: int(x))
df_train["round_trip"] = df_train["round_trip"].apply(lambda x: int(x))

In [11]:
le = LabelEncoder().fit(df_train["passholder_type"])
df_train["passholder_type_target"] = le.transform(df_train["passholder_type"])
df_train = df_train.drop(["passholder_type"], axis=1)

In [12]:
df_train

,duration,start_station,end_station,one_way,round_trip,passholder_type_target
0,35,4127,4127,0,1,5
1,32,3057,3062,1,0,5
2,6,3022,3062,1,0,2
3,138,4304,4311,1,0,3
4,14,4266,4443,1,0,2
...,...,...,...,...,...,...
697419,17,3032,4220,1,0,2
697420,9,3036,3014,1,0,2
697421,6,3032,3007,1,0,1
697422,3,3067,3006,1,0,5


In [13]:
# los datos estan desbalanceados
idx = pd.Index(df_train["passholder_type_target"])
idx.value_counts()

2    394769
5    212426
3     44455
0     34092
1     11604
4        78
Name: passholder_type_target, dtype: int64

In [14]:
x = df_train.iloc[:, :-1].to_numpy()

In [ ]:
# scaler = StandardScaler().fit(x)
# x = scaler.transform(x)

In [15]:
y = df_train["passholder_type_target"].to_numpy()

In [16]:
skf = StratifiedKFold(n_splits = 3)

In [17]:
inicio = time.time()

acc_rf = []
f1_rf = []
acc_mlp = []
f1_mlp = []
i = 0
for iTrain, jTest in skf.split(x, y):
    print("\nIteracion:", i + 1)
    x_train, x_test = x[iTrain], x[jTest]
    y_train, y_test = y[iTrain], y[jTest]
    
    print("RandomForestClassifier")
    clf_rf = RandomForestClassifier()
    clf_rf.fit(x_train, y_train)
    acc_rf.append(clf_rf.score(x_test, y_test))
    y_pred = clf_rf.predict(x_test)
    f1_rf.append(f1_score(y_test, y_pred, average='macro'))
    print(confusion_matrix(y_test, y_pred))

    print("MLPClassifier")
    clf_mlp = MLPClassifier()
    clf_mlp.fit(x_train, y_train)
    acc_mlp.append(clf_mlp.score(x_test, y_test))
    y_pred = clf_mlp.predict(x_test)
    f1_mlp.append(f1_score(y_test, y_pred, average='macro'))
    print(confusion_matrix(y_test, y_pred))
    
    i += 1
print("RandomForestClassifier accuracy: ", acc_rf)
print("RandomForestClassifier f1 socre: ", f1_rf)

print("MLPClassifier accuracy: ", acc_mlp)
print("MLPClassifier f1 socre: ", f1_mlp)

print("Elapsed:", time.time() - inicio)


Iteracion: 1
RandomForestClassifier
[[  2390     58   7440    187      1   1288]
 [    34    275   2863     30      0    666]
 [  2040    453 110594   2047      2  16454]
 [   185     31   4871   2302      3   7426]
 [     5      0      7      1      0     13]
 [   797    250  25332   3395      4  41031]]
MLPClassifier
[[     0      0  10812      0      0    552]
 [     0      0   3634      0      0    234]
 [     0      0 122844      0      0   8746]
 [     0      0   7867      0      0   6951]
 [     0      0     23      0      0      3]
 [     0      0  39363      0      0  31446]]

Iteracion: 2
RandomForestClassifier
[[  2433     46   7460    211      1   1213]
 [    45    256   2897     41      0    629]
 [  2107    462 110676   2070      4  16270]
 [   162     39   4766   2358      0   7494]
 [     2      0      9      6      0      9]
 [   773    283  25434   3250      0  41069]]
MLPClassifier
[[     0      0  10223      0      0   1141]
 [     0      0   3598      0      0    

In [18]:
print("RandomForestClassifier")
print("Mean Accuracy: %.2f" % np.mean(acc_rf))
print("std: (+/- %.2f)" % np.std(acc_rf))
print("Mean F1 Score: %.2f" % np.mean(f1_rf))
print("std: (+/- %.2f)" % np.std(f1_rf))

print("MLPClassifier")
print("Mean Accuracy: %.2f" % np.mean(acc_mlp))
print("std: (+/- %.2f)" % np.std(acc_mlp))
print("Mean F1 Score: %.2f" % np.mean(f1_mlp))
print("std: (+/- %.2f)" % np.std(f1_mlp))

RandomForestClassifier
Mean Accuracy: 0.67
std: (+/- 0.00)
Mean F1 Score: 0.33
std: (+/- 0.00)
MLPClassifier
Mean Accuracy: 0.66
std: (+/- 0.00)
Mean F1 Score: 0.22
std: (+/- 0.00)
